Code to import WRDS data to PostgreSQL
=========

This repository contains code to pull together data from [WRDS](https://wrds-web.wharton.upenn.edu/wrds/).

The code will only work if you have access to WRDS and to the data in question.

## Requirements

### 1. Git

While not strictly necessary to use the scripts here, [Git](https://git-scm.com/downloads) likely makes it easier to download and to update.

I keep all Git repositories in `~/git`. So to get this repository, I could do:

```
cd ~/git
git clone https://github.com/iangow/wrds_pg.git
```

This will create a copy of the repository in `~/git/wrds_pg`.
Note that one can get updates to the repository by going to the directory and "pulling" the latest code:

```
cd ~/git/wrds_pg
git pull
```

Alternatively, I think you could fork the repository on GitHub and then clone. 
I think that cloning using the SSH URL (e.g., `git@github.com:iangow/wrds_pg.git`) is necessary for Git pulling and pushing to work well in RStudio.

### 2. PostgreSQL

You should have a PostgreSQL database to store the data.
There are also some data dependencies in that some scripts assume the existence of other data in the database.
Also, I assume the existence of role `wrds` and `wrds_access` (SQL like `CREATE ROLE wrds` works to add these if they are absent).

### 3. Environment variables

I am migrating the scripts, etc., from using hard-coded values (e.g., my WRDS ID `iangow`) to using environment variales. 
Environment variables that I use include:

- `PGDATABASE`: The name of the PostgreSQL database you use.
- `PGUSER`: Your username on the PostgreSQL database.
- `PGHOST`: Where the PostgreSQL database is to be found (this will be `localhost` if its on the same machine as you're running the code on)
- `WRDS_ID`: Your [WRDS](https://wrds-web.wharton.upenn.edu/wrds/) ID.

I set these environment variables in `~/.profile`:

```
export PGHOST="localhost"
export PGDATABASE="crsp"
export WRDS_ID="iangow"
export PGUSER="igow"
```

### 4. A WRDS ID

Note that I use public-key authentication to access WRDS. Following hints taken from [here](http://www.debian-administration.org/articles/152), I set up a public key. I then copied that key to the WRDS server from the terminal on my computer. (Note that this code assumes you have a directory `.ssh` in your home directory. If not, log into WRDS via SSH, then type `mkdir ~/.ssh` to create this.) Here's code to create the key and send it to WRDS (for me):

```
ssh-keygen -t rsa
cat ~/.ssh/id_rsa.pub | ssh iangow@wrds-cloud.wharton.upenn.edu "cat >> ~/.ssh/authorized_keys"
```

I use an empty passphrase in setting up my key so that the scripts can run without user intervention.

In [1]:
from sqlalchemy import create_engine
import os
dbname = os.getenv("PGDATABASE")
host = os.getenv("PGHOST", "localhost")
user = os.getenv("PGUSER", os.getlogin())
wrds_id = "iangow" # os.getenv("WRDS_ID")
dbname = engine = create_engine("postgresql://" + user + "@" + host + "/" + dbname)

from wrds_fetch import wrds_update, set_table_comment
print(wrds_id)

iangow


In [2]:
schema = "crsp"
table_name = "dsi"

In [3]:
from wrds_fetch import wrds_update
wrds_update(table_name, schema, engine, wrds_id, force=True)

Forcing update based on user request.


True

In [4]:
import pandas as pd
df = pd.read_sql("""SELECT * FROM "%s"."%s" LIMIT 6""" % (schema, table_name), engine)
df

,date,vwretd,vwretx,ewretd,ewretx,sprtrn,spindx,totval,totcnt,usdval,usdcnt
0,1925-12-31,NaN,NaN,NaN,NaN,None,None,27487487.2,503,NaN,NaN
1,1926-01-02,0.005689,0.005689,0.009516,0.009516,None,None,27600296.6,497,27366755.7,494.0
2,1926-01-04,0.000706,0.000706,0.005780,0.005780,None,None,27577892.1,502,27479721.6,495.0
3,1926-01-05,-0.004821,-0.004867,-0.001927,-0.002030,None,None,27530212.5,501,27561757.1,499.0
4,1926-01-06,-0.000423,-0.000427,0.001182,0.001155,None,None,27618772.6,505,27526837.5,500.0
5,1926-01-07,0.004988,0.004953,0.008453,0.008384,None,None,27690393.8,504,27553072.6,504.0
